In [1]:
import DeepFM
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Activation, Layer
import random

In [2]:
dim, feature_names, feature_sizes, batch_size, deep_dense_out_dim = 3, ["a", "b", "c"], [3, 2, 2], 2, [3, 1]

In [3]:
model = DeepFM.DeepFM(dim, feature_names, feature_sizes, batch_size, deep_dense_out_dim)

In [4]:
# trigger the lazy tensors
[x(0) for x in model.features_embeds]
model.fm_bias(tf.constant([1.0, 2.0, 3.0]))
[x(0) for x in model.fm_dense]

# deep part input size is (batch_size, n_fea*dim)
out = tf.constant([[random.random() for _ in range(len(feature_names)*dim)]])
for l in model.deep_dense_layers:
    out = l(out)

In [5]:
# model parameters
for i, embed_layer in enumerate(model.features_embeds):
    print("------- feature-{} embedding vectors -------".format(i))
    print(embed_layer.weights[0].numpy())
    print()

print("------- fm dense -------")
for i, fea_dense_w_lookup in enumerate(model.fm_dense):
    print("------ fm feature-{} dense w lookup ------".format(i))
    print(fea_dense_w_lookup.weights[0].numpy())

print("------- fm bias -------")
print(model.fm_bias.weights[0].numpy())


------- feature-0 embedding vectors -------
[[ 0.01288019  0.03272445 -0.01649339]
 [ 0.04020175 -0.00628924  0.00319413]
 [ 0.00949966  0.02584275  0.02168764]]

------- feature-1 embedding vectors -------
[[ 0.0129966  -0.02178497  0.04210693]
 [-0.01770421 -0.00483744  0.00950954]]

------- feature-2 embedding vectors -------
[[ 0.04772587  0.04583821 -0.0478565 ]
 [-0.00487811 -0.01407316 -0.02572885]]

------- fm dense -------
------ fm feature-0 dense w lookup ------
[[0.01222704]
 [0.04848984]
 [0.04873163]]
------ fm feature-1 dense w lookup ------
[[-0.01674583]
 [-0.04932194]]
------ fm feature-2 dense w lookup ------
[[0.00101062]
 [0.00623454]]
------- fm bias -------
[-0.084661]


In [6]:
print("------- deep dense -------")
for i, dense_layer in enumerate(model.deep_dense_layers):
    print("---------- deep dense-{} layer ----------".format(i))
    print("weights:")
    print(dense_layer.weights[0].numpy())
    print("bias:")
    print(dense_layer.weights[1].numpy())

------- deep dense -------
---------- deep dense-0 layer ----------
weights:
[[-0.06075704 -0.45923695  0.1717745 ]
 [-0.6454746  -0.00456971  0.36452907]
 [ 0.33800882  0.36106378 -0.64581275]
 [ 0.26727337 -0.03098249  0.6380921 ]
 [-0.3135201   0.33219272  0.23493767]
 [-0.04501891 -0.3936964  -0.19730657]
 [-0.5929896   0.5451148   0.63756806]
 [ 0.50880164 -0.16037196 -0.37451488]
 [ 0.47448808  0.37647945 -0.16552669]]
bias:
[0. 0. 0.]
---------- deep dense-1 layer ----------
weights:
[[ 0.88652265]
 [-0.04007936]
 [ 0.16886973]]
bias:
[0.]


In [7]:
# test data
samples = [[0, 1, 0], [1, 0, 1]]
dataset = tf.data.Dataset.from_tensor_slices(samples)
dataset = dataset.batch(batch_size)

for batch in dataset:
    batch_pre = model(batch)
    print("pred")
    print(batch_pre)

fm part
[-0.11990618 -0.04691842]
deep part
feature embeds concated:
[[ 0.01288019  0.03272445 -0.01649339 -0.01770421 -0.00483744  0.00950954
   0.04772587  0.04583821 -0.0478565 ]
 [ 0.04020175 -0.00628924  0.00319413  0.0129966  -0.02178497  0.04210693
  -0.00487811 -0.01407316 -0.02572885]]

deep layer 0 out
[[0.         0.         0.03166636]
 [0.         0.         0.00383644]]

deep layer 1 out
[[0.00534749]
 [0.00064786]]

[0.00534749 0.00064786]

feature embeds concated:
[[ 0.01288019  0.03272445 -0.01649339 -0.01770421 -0.00483744  0.00950954
   0.04772587  0.04583821 -0.0478565 ]
 [ 0.04020175 -0.00628924  0.00319413  0.0129966  -0.02178497  0.04210693
  -0.00487811 -0.01407316 -0.02572885]]

deep layer 0 out
[[0.         0.         0.03166636]
 [0.         0.         0.00383644]]

deep layer 1 out
[[0.00534749]
 [0.00064786]]

pred
tf.Tensor([0.47139162 0.48843443], shape=(2,), dtype=float32)


In [16]:

model.deep_dense_layers[0](tf.constant([[ 0.01288019,  0.03272445, -0.01649339, -0.01770421, -0.00483744,  0.00950954,0.04772587,  0.04583821, -0.0478565 ]])).numpy()

array([[0.        , 0.        , 0.03166636]], dtype=float32)

In [17]:
tf.matmul(tf.constant([[ 0.01288019,  0.03272445, -0.01649339, -0.01770421, -0.00483744,  0.00950954,0.04772587,  0.04583821, -0.0478565 ]]), model.deep_dense_layers[0].weights[0])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[-0.05880934, -0.01617415,  0.03166636]], dtype=float32)>

In [14]:
model.deep_dense_layers[0].weights[0]

<tf.Variable 'deep_dense_0/kernel:0' shape=(9, 3) dtype=float32, numpy=
array([[-0.06075704, -0.45923695,  0.1717745 ],
       [-0.6454746 , -0.00456971,  0.36452907],
       [ 0.33800882,  0.36106378, -0.64581275],
       [ 0.26727337, -0.03098249,  0.6380921 ],
       [-0.3135201 ,  0.33219272,  0.23493767],
       [-0.04501891, -0.3936964 , -0.19730657],
       [-0.5929896 ,  0.5451148 ,  0.63756806],
       [ 0.50880164, -0.16037196, -0.37451488],
       [ 0.47448808,  0.37647945, -0.16552669]], dtype=float32)>

In [15]:
model.deep_dense_layers[0].weights[1]

<tf.Variable 'deep_dense_0/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>